In [136]:
import os 
import json
import pandas as pd
import traceback
import PyPDF2

In [9]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain.chat_models import ChatOpenAI

In [11]:
key = os.getenv('OPENAI_API_KEY')

In [12]:
llm = ChatOpenAI(openai_api_key = key,model_name = 'gpt-3.5-turbo',temperature = 0.3)

In [19]:
from langchain import PromptTemplate, LLMChain, OpenAI
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback

In [55]:
RESPONSE_JSON = {
    
    '1' : {
        
        'question' : 'the mcq question',
        'options' : {
            
            'A' : 'choice here',
            'B' : 'choice here',
            'C' : 'choice here',
            'D' : 'choice here'
        },
        'correct' : 'correct choice'
    },
    
    '2' : {
        
        'question' : 'the mcq question',
        'options' : {
            
            'A' : 'choice here',
            'B' : 'choice here',
            'C' : 'choice here',
            'D' : 'choice here'
        },
        'correct' : 'correct choice'
    },
    
    '3' : {
        
        'question' : 'the mcq question',
        'options' : {
            
            'A' : 'choice here',
            'B' : 'choice here',
            'C' : 'choice here',
            'D' : 'choice here'
        },
        'correct' : 'correct choice'
    },
    
    '4' : {
        
        'question' : 'the mcq question',
        'options' : {
            
            'A' : 'choice here',
            'B' : 'choice here',
            'C' : 'choice here',
            'D' : 'choice here'
        },
        'correct' : 'correct choice'
    }
}

In [87]:
Template = '''

You are a helpful MCQ Generater and u have been given a text : {text},

You have to generate {number} mcq questions on {subject} and 
level of difficulty should be {level} .

The response should be kept the way it is given in RESPONSE_JSON and use it as guide

### RESPONSE_JSON

{response_json}'''

In [88]:
prompt = PromptTemplate(
    
    input_variables = ['text','number','subject','level','response_json'],
    template = Template
)

In [89]:
quiz_chain = LLMChain(llm = llm,prompt = prompt,output_key = 'quiz')

In [90]:
Template2 = '''

Quiz : {quiz} ,

U are an english grammerian and writer 
Task : Ur task is to analyse the complexity of the quiz questions in the given quiz 
based on the complexity level you are seeing in the quiz for the given subject : {subject},

If the quiz doesn't fit with the cognitve abiltiy of the student then just change the 
questions in the quiz and difficulty level of the quiz to make it fit
'''

In [91]:
# to check the complexity of the quiz

complex_prompt = PromptTemplate(
    
    input_variables = ['quiz','subject'],
    template = Template2
)

In [92]:
complex_chain = LLMChain(llm = llm, prompt = complex_prompt, output_key = 'complexity')

In [93]:
final_chain = SequentialChain(chains = [quiz_chain, complex_chain], input_variables = ['text','number','subject','level','response_json'],output_variables = ['quiz','complexity'])

In [94]:
file_path = 'D:\Project\data.txt'

In [95]:
with open(file_path,'r') as file:
    file_data = file.read()

In [96]:
file_data

'Levels of Thinking\nCarol Gilligan states that the post-conventional level of moral thinking can be dealt based on the\xa0two types of thinking. Gilligan’s theory is based on the two main ideas, the care-based morality (usually found in women) and the justice-based morality (usually found in men).\n\nCare-based Morality\nCare-based morality is the kind of thinking found in women. This is based on the following principles.\nMore emphasis is given to inter-connected relationships and universality.\nActing justly focuses on avoidance of violence.\nWomen with this are usually interested in helping others.\nMore common in girls because of their connections to their mothers.\nBecause girls remain connected to their mothers, they are less inclined to worry about issues of fairness.\n\nJustice-based Morality\nJustice-based morality is the kind of thinking found in men. This is based on the following principles.\nThey view the world as being composed of autonomous individuals who interact with

our RESPONSE_JSON is in dictionary format so we have to change it into the json format

In [97]:
json.dumps(RESPONSE_JSON)

'{"1": {"question": "the mcq question", "options": {"A": "choice here", "B": "choice here", "C": "choice here", "D": "choice here"}, "correct": "correct choice"}, "2": {"question": "the mcq question", "options": {"A": "choice here", "B": "choice here", "C": "choice here", "D": "choice here"}, "correct": "correct choice"}, "3": {"question": "the mcq question", "options": {"A": "choice here", "B": "choice here", "C": "choice here", "D": "choice here"}, "correct": "correct choice"}, "4": {"question": "the mcq question", "options": {"A": "choice here", "B": "choice here", "C": "choice here", "D": "choice here"}, "correct": "correct choice"}}'

In [98]:
number = 5
subject = 'english'
level = 'intermediate'

### get_openai_callback 

Tell about the tokens which are used : prompt token, completion token and price

In [100]:
with get_openai_callback() as cb :
    
    response = final_chain(
        {
            'text' : file_data,
            'number' : number,
            'subject' : subject,
            'level' : level,
            'response_json' : json.dumps(RESPONSE_JSON)
        }
    )

In [101]:
cb

Tokens Used: 1733
	Prompt Tokens: 1099
	Completion Tokens: 634
Successful Requests: 2
Total Cost (USD): $0.0029165

In [102]:
response

{'text': 'Levels of Thinking\nCarol Gilligan states that the post-conventional level of moral thinking can be dealt based on the\xa0two types of thinking. Gilligan’s theory is based on the two main ideas, the care-based morality (usually found in women) and the justice-based morality (usually found in men).\n\nCare-based Morality\nCare-based morality is the kind of thinking found in women. This is based on the following principles.\nMore emphasis is given to inter-connected relationships and universality.\nActing justly focuses on avoidance of violence.\nWomen with this are usually interested in helping others.\nMore common in girls because of their connections to their mothers.\nBecause girls remain connected to their mothers, they are less inclined to worry about issues of fairness.\n\nJustice-based Morality\nJustice-based morality is the kind of thinking found in men. This is based on the following principles.\nThey view the world as being composed of autonomous individuals who inte

In [104]:
quiz = response.get('quiz')

In [109]:
quiz = json.loads(quiz)

In [116]:
quiz.items()

dict_items([('1', {'question': 'According to Carol Gilligan, care-based morality is usually found in ________.', 'options': {'A': 'men', 'B': 'women', 'C': 'boys', 'D': 'girls'}, 'correct': 'B'}), ('2', {'question': 'What is the main focus of care-based morality?', 'options': {'A': 'Avoidance of violence', 'B': 'Inter-connected relationships and universality', 'C': 'Protecting individuality', 'D': 'Differentiation between themselves and their mothers'}, 'correct': 'B'}), ('3', {'question': "According to Gilligan's theory, justice-based morality is usually found in ________.", 'options': {'A': 'men', 'B': 'women', 'C': 'boys', 'D': 'girls'}, 'correct': 'A'}), ('4', {'question': 'What is the main focus of justice-based morality?', 'options': {'A': 'Avoidance of violence', 'B': 'Inter-connected relationships and universality', 'C': 'Protecting individuality', 'D': 'Differentiation between themselves and their mothers'}, 'correct': 'C'}), ('5', {'question': 'Which perspective views a probl

In [115]:
quiz['1'].items()

dict_items([('question', 'According to Carol Gilligan, care-based morality is usually found in ________.'), ('options', {'A': 'men', 'B': 'women', 'C': 'boys', 'D': 'girls'}), ('correct', 'B')])

In [119]:
print(7," ",2)

7   2


In [124]:
'|'.join(['a','k','r','t'])

'a|k|r|t'

In [128]:

table = []

for key,value in quiz.items():
    
    print(key," | ",value)
    
    ques = value['question']
    options = ' | '.join( 
        [
            f"{option} : {options}"
            for option, options in value['options'].items()
            
            ]
        )
    correct = value['correct']
    table.append({'ques':ques,'options' : options, 'correct' : correct})


     
    

1  |  {'question': 'According to Carol Gilligan, care-based morality is usually found in ________.', 'options': {'A': 'men', 'B': 'women', 'C': 'boys', 'D': 'girls'}, 'correct': 'B'}
2  |  {'question': 'What is the main focus of care-based morality?', 'options': {'A': 'Avoidance of violence', 'B': 'Inter-connected relationships and universality', 'C': 'Protecting individuality', 'D': 'Differentiation between themselves and their mothers'}, 'correct': 'B'}
3  |  {'question': "According to Gilligan's theory, justice-based morality is usually found in ________.", 'options': {'A': 'men', 'B': 'women', 'C': 'boys', 'D': 'girls'}, 'correct': 'A'}
4  |  {'question': 'What is the main focus of justice-based morality?', 'options': {'A': 'Avoidance of violence', 'B': 'Inter-connected relationships and universality', 'C': 'Protecting individuality', 'D': 'Differentiation between themselves and their mothers'}, 'correct': 'C'}
5  |  {'question': 'Which perspective views a problem as a conflict bet

In [129]:
table

[{'ques': 'According to Carol Gilligan, care-based morality is usually found in ________.',
  'options': 'A : men | B : women | C : boys | D : girls',
  'correct': 'B'},
 {'ques': 'What is the main focus of care-based morality?',
  'options': 'A : Avoidance of violence | B : Inter-connected relationships and universality | C : Protecting individuality | D : Differentiation between themselves and their mothers',
  'correct': 'B'},
 {'ques': "According to Gilligan's theory, justice-based morality is usually found in ________.",
  'options': 'A : men | B : women | C : boys | D : girls',
  'correct': 'A'},
 {'ques': 'What is the main focus of justice-based morality?',
  'options': 'A : Avoidance of violence | B : Inter-connected relationships and universality | C : Protecting individuality | D : Differentiation between themselves and their mothers',
  'correct': 'C'},
 {'ques': 'Which perspective views a problem as a conflict between two individual groups?',
  'options': 'A : Care-based pe

In [133]:
quiz = pd.DataFrame(table)
quiz

,ques,options,correct
0,"According to Carol Gilligan, care-based morali...",A : men | B : women | C : boys | D : girls,B
1,What is the main focus of care-based morality?,A : Avoidance of violence | B : Inter-connecte...,B
2,"According to Gilligan's theory, justice-based ...",A : men | B : women | C : boys | D : girls,A
3,What is the main focus of justice-based morality?,A : Avoidance of violence | B : Inter-connecte...,C
4,Which perspective views a problem as a conflic...,A : Care-based perspective | B : Justice-based...,B


In [135]:
quiz.to_csv("PEHV.csv")